# Definiton of all measurements

## Denition of KeyPoints

In [ ]:
# 'nose',                                   pos 0
#'left_eye', 'right_eye',                   pos 1 / 2
# 'left_ear', 'right_ear',                  pos 3 / 4
# 'left_shoulder', 'right_shoulder',        pos 5 / 6
# 'left_elbow', 'right_elbow',              pos 7 / 8
# 'left_wrist', 'right_wrist',              pos 9 / 10
# 'left_hip', 'right_hip',                  pos 11 / 12
# 'left_knee', 'right_knee',                pos 13 / 14
# 'left_ankle', 'right_ankle',              pos 15 / 16

#Format of 17 keypoints matrix for 1 image: 
#17 / 3 [y,x,likelyhood] 


In [116]:
import numpy as np
import math

In [117]:
image_front_T_kp = [[0.17349216, 0.56427723, 0.5236098 ],
        [0.16178887, 0.57046986, 0.52236605],
        [0.16322146, 0.55656576, 0.6874031 ],
        [0.17638007, 0.5828533 , 0.72497916],
        [0.17946883, 0.55041116, 0.76183915],
        [0.24724907, 0.60664934, 0.6900308 ],
        [0.2661889 , 0.5356395 , 0.7871611 ],
        [0.3592037 , 0.61628497, 0.5966688 ],
        [0.3696247 , 0.549422  , 0.5074349 ],
        [0.4646902 , 0.6258748 , 0.59548384],
        [0.29332668, 0.5630101 , 0.47110534],
        [0.42144495, 0.6031153 , 0.84182334],
        [0.42856565, 0.559704  , 0.79291236],
        [0.5857684 , 0.6084723 , 0.7321706 ],
        [0.5991622 , 0.5660539 , 0.78673285],
        [0.72647244, 0.61776626, 0.7692236 ],
        [0.7399327 , 0.59163785, 0.62649405]]
image_front_T_kp = np.array(image_front_T_kp)

In [118]:
image_front_I_kp = [[0.13626342, 0.57170576, 0.5406235 ],
       [0.12581952, 0.58031356, 0.5950187 ],
       [0.12306002, 0.5641533 , 0.5945765 ],
       [0.1330445 , 0.58676374, 0.46940276],
       [0.1319304 , 0.55489534, 0.48606545],
       [0.19684711, 0.6048043 , 0.37372732],
       [0.19630507, 0.53479004, 0.30953246],
       [0.26008302, 0.6065161 , 0.42267993],
       [0.25835255, 0.51460385, 0.49753478],
       [0.23563783, 0.59914243, 0.28201967],
       [0.25673127, 0.49728453, 0.4280582 ],
       [0.3964715 , 0.5841227 , 0.5168887 ],
       [0.3921252 , 0.54193866, 0.47156578],
       [0.5595846 , 0.58796036, 0.54856455],
       [0.5583141 , 0.54261553, 0.52169836],
       [0.7302867 , 0.5933529 , 0.4042293 ],
       [0.7241063 , 0.5598221 , 0.30607915]]
image_front_I_kp = np.array(image_front_I_kp)

In [119]:
image_profile_kp = [[0.17119896, 0.55431795, 0.65762174],
       [0.1641411 , 0.5632325 , 0.5738114 ],
       [0.16227084, 0.5482719 , 0.6999848 ],
       [0.17950638, 0.574966  , 0.570612  ],
       [0.17202213, 0.54290915, 0.57458735],
       [0.23494339, 0.56739473, 0.415179  ],
       [0.24977106, 0.55462873, 0.45368198],
       [0.3402242 , 0.5621322 , 0.44017628],
       [0.3311993 , 0.5404615 , 0.28933096],
       [0.4339265 , 0.5579535 , 0.62782705],
       [0.29925185, 0.50375867, 0.4816786 ],
       [0.43029714, 0.57282275, 0.57726043],
       [0.43196964, 0.5522737 , 0.5526497 ],
       [0.60913986, 0.55608445, 0.68798673],
       [0.60523015, 0.5404867 , 0.48570338],
       [0.75797564, 0.5934522 , 0.49470276],
       [0.7497481 , 0.56782985, 0.51544404]]
image_profile_kp = np.array(image_profile_kp)

In [120]:
#ADDITIONAL KEYPOINTS on picture FRONT_T / FRONT I

#left biceps, right biceps                      pos 17 / 18

def additional_kp_front(image_front_T_kp, image_front_I_kp):
    #Biceps - Image front T
    #left biceps pos 17
    l_x11 = image_front_T_kp[7][1] +abs(image_front_T_kp[7][1] - image_front_T_kp[5][1])/2
    l_y11 = (image_front_T_kp[7][0] + image_front_T_kp[5][0])/2
    #right biceps pos 18
    r_x11 = image_front_T_kp[6][1] + abs(image_front_T_kp[8][1]-image_front_T_kp[6][1])
    r_y11 = (image_front_T_kp[8][0] + image_front_T_kp[6][0])/2
    
    #Biceps - Image front I
    #left biceps pos 17
    l_x12 = abs(image_front_T_kp[7][1] + image_front_T_kp[5][1])/2
    l_y12 = image_front_T_kp[5][0] + abs(image_front_T_kp[7][0] - image_front_T_kp[5][0])/2
    #right biceps pos 18
    r_x12 = abs(image_front_T_kp[8][1]+image_front_T_kp[6][1])/2
    r_y12 = image_front_T_kp[6][0] + (image_front_T_kp[8][0] + image_front_T_kp[6][0])/2
    

    #left chest, right chest                      pos 19 / 20
    #left chest pos 19 
    l_x2 = image_front_T_kp[5][1] + abs(image_front_T_kp[6][1] - image_front_T_kp[5][1]) * 1/4   #considering chest "x" located  at 1/4 vs soulders
    l_y2 = image_front_T_kp[5][0] + abs(image_front_T_kp[11][0] - image_front_T_kp[5][0]) * 1/3  #considering chest "y" located  at 1/3 vs shoulders/hips
    #right chest pos 20
    r_x2 = image_front_T_kp[5][1] + abs(image_front_T_kp[6][1] - image_front_T_kp[5][1]) * 3/4   #considering chest "x" located  at 3/4 vs soulders
    r_y2 = image_front_T_kp[6][0] + abs(image_front_T_kp[12][0] - image_front_T_kp[6][0]) * 1/3  #considering chest "y" located  at 1/3 vs shoulders/hips
    
    #left waist, right waist                       pos 21 / 22
    #left waist pos 21
    l_x3 = l_x2                                                                                   #considering chest "x" located  at 1/4 vs soulders
    l_y3 = image_front_T_kp[5][0] + abs(image_front_T_kp[11][0] - image_front_T_kp[5][0]) * 2/3  #considering chest "y" located  at 2/3 vs shoulders/hi
    #right waist pos 22
    r_x3 = r_x2                                                                                   #considering chest "x" located  at 3/4 vs soulders
    r_y3 = image_front_T_kp[6][0] + abs(image_front_T_kp[12][0] - image_front_T_kp[6][0]) * 2/3  #considering chest "y" located  at 2/3 vs shoulders/hips
    
    #append new keypoints into matrix_kp front T, front I
    new_matrix_kp_T = np.vstack((image_front_T_kp, np.array([l_y11, l_x11, 0]), np.array([r_y11, r_x11, 0]), \
                              np.array([l_y2, l_x2, 0]), np.array([r_y2, r_x2, 0]), \
                              np.array([l_y3, l_x3, 0]), np.array([r_y3, r_x3, 0])))
    
    new_matrix_kp_I = np.vstack((image_front_I_kp, np.array([l_y12, l_x12, 0]), np.array([r_y12, r_x12, 0]), \
                              np.array([l_y2, l_x2, 0]), np.array([r_y2, r_x2, 0]), \
                              np.array([l_y3, l_x3, 0]), np.array([r_y3, r_x3, 0])))

    
    return new_matrix_kp_T, new_matrix_kp_I


In [121]:
additional_kp_front(image_front_T_kp, image_front_I_kp)

(array([[0.17349216, 0.56427723, 0.5236098 ],
        [0.16178887, 0.57046986, 0.52236605],
        [0.16322146, 0.55656576, 0.6874031 ],
        [0.17638007, 0.5828533 , 0.72497916],
        [0.17946883, 0.55041116, 0.76183915],
        [0.24724907, 0.60664934, 0.6900308 ],
        [0.2661889 , 0.5356395 , 0.7871611 ],
        [0.3592037 , 0.61628497, 0.5966688 ],
        [0.3696247 , 0.549422  , 0.5074349 ],
        [0.4646902 , 0.6258748 , 0.59548384],
        [0.29332668, 0.5630101 , 0.47110534],
        [0.42144495, 0.6031153 , 0.84182334],
        [0.42856565, 0.559704  , 0.79291236],
        [0.5857684 , 0.6084723 , 0.7321706 ],
        [0.5991622 , 0.5660539 , 0.78673285],
        [0.72647244, 0.61776626, 0.7692236 ],
        [0.7399327 , 0.59163785, 0.62649405],
        [0.30322638, 0.62110279, 0.        ],
        [0.3179068 , 0.549422  , 0.        ],
        [0.30531436, 0.6244018 , 0.        ],
        [0.32031448, 0.65990672, 0.        ],
        [0.36337966, 0.6244018 , 0

In [123]:
#ADDITIONAL KEYPOINTS on picture PROFILE

#left biceps, right biceps                      pos 17 / 18

def additional_kp_profile(image_profile_kp):
    #left biceps pos 17
    l_x1 = (image_profile_kp[7][1] + image_profile_kp[5][1])/2                            #average on x
    l_y1 = image_front_T_kp[5][0] + (image_front_T_kp[7][0] - image_front_T_kp[5][0])/2   #considering biceps is at half shoulder/elbow
    #right biceps pos 18
    r_x1 = abs(image_front_T_kp[8][1] + image_front_T_kp[6][1])/2                         #average on x
    r_y1 = image_front_T_kp[6][0] + (image_front_T_kp[8][0] + image_front_T_kp[6][0])/2   #considering biceps is at half shoulder/elbow

    #left chest, right chest                      pos 19 / 20
    #left chest pos 19 
    l_x2 = l_x1
    l_y2 = image_front_T_kp[5][0] + abs(image_front_T_kp[11][0] - image_front_T_kp[5][0]) * 1/3  #considering chest "y" located  at 1/3 vs shoulders/hips
    #right chest pos 20
    r_x2 = r_x1 
    r_y2 = image_front_T_kp[6][0] + abs(image_front_T_kp[12][0] - image_front_T_kp[6][0]) * 1/3  #considering chest "y" located  at 1/3 vs shoulders/hips

    #left waist, right waist                       pos 21 / 22
    #left waist pos 21
    l_x3 = l_x1
    l_y3 = image_front_T_kp[5][0] + abs(image_front_T_kp[11][0] - image_front_T_kp[5][0]) * 2/3  #considering chest "y" located  at 2/3 vs shoulders/hi
    #right waist pos 22
    r_x3 = r_x1
    r_y3 = image_front_T_kp[6][0] + abs(image_front_T_kp[12][0] - image_front_T_kp[6][0]) * 2/3  #considering chest "y" located  at 2/3 vs shoulders/hips
    
    #append new keypoints into matrix_kp
    new_matrix_kp_profile = np.vstack((image_front_T_kp, np.array([l_y1, l_x1, 0]), np.array([r_y1, r_x1, 0]), \
                              np.array([l_y2, l_x2, 0]), np.array([r_y2, r_x2, 0]), \
                              np.array([l_y3, l_x3, 0]), np.array([r_y3, r_x3, 0])))

    
    return new_matrix_kp_profile


In [124]:
additional_kp_profile(image_profile_kp)

array([[0.17349216, 0.56427723, 0.5236098 ],
       [0.16178887, 0.57046986, 0.52236605],
       [0.16322146, 0.55656576, 0.6874031 ],
       [0.17638007, 0.5828533 , 0.72497916],
       [0.17946883, 0.55041116, 0.76183915],
       [0.24724907, 0.60664934, 0.6900308 ],
       [0.2661889 , 0.5356395 , 0.7871611 ],
       [0.3592037 , 0.61628497, 0.5966688 ],
       [0.3696247 , 0.549422  , 0.5074349 ],
       [0.4646902 , 0.6258748 , 0.59548384],
       [0.29332668, 0.5630101 , 0.47110534],
       [0.42144495, 0.6031153 , 0.84182334],
       [0.42856565, 0.559704  , 0.79291236],
       [0.5857684 , 0.6084723 , 0.7321706 ],
       [0.5991622 , 0.5660539 , 0.78673285],
       [0.72647244, 0.61776626, 0.7692236 ],
       [0.7399327 , 0.59163785, 0.62649405],
       [0.30322638, 0.56476346, 0.        ],
       [0.5840957 , 0.54253075, 0.        ],
       [0.30531436, 0.56476346, 0.        ],
       [0.32031448, 0.54253075, 0.        ],
       [0.36337966, 0.56476346, 0.        ],
       [0.

In [125]:
#Convert Matrix Keypoints coordinates in x,y to pixels(contouring matrix)
def convert_kp_matrix_front_T(matrix_kp_front_T, image_h, image_l):
    x = np.round(matrix_kp_front_T[: , 1] * image_l)
    y = np.round(matrix_kp_front_T[: , 0] * image_h)
    matrix_kp_converted = list(zip(y,x))
    return matrix_kp_converted

def convert_kp_matrix_front_I(matrix_kp_front_I, image_h, image_l):
    x = np.round(matrix_kp_front_I[: , 1] * image_l)
    y = np.round(matrix_kp_front_I[: , 0] * image_h)
    matrix_kp_converted = list(zip(y,x))
    return matrix_kp_converted

def convert_kp_matrix_profile(matrix_kp_profile, image_h, image_l):
    x = np.round(matrix_kp_profile[: , 1] * image_l)
    y = np.round(matrix_kp_profile[: , 0] * image_h)
    matrix_kp_converted = list(zip(y,x))
    return matrix_kp_converted

# Define ratio between real measure(in mm) vs pixel position
def ratio_real_vs_pixel(real_height, matrix_image_contouring, image_h):
    list_sum_y=[]
    #sum of all "1" for each x to get the sum max ie. height
    for i in range(0,image_h):
        list_sum_y.append(np.sum(matrix_image_contouring[: , i]))
    ratio_mm_px = real_height / max(list_sum_y)                            #ratio real height (in mm) / max sum of "1"
    return ratio_mm_px

In [103]:
image_front_T_kp = additional_kp(image_front_T_kp)

0.009635630000000006 0.303226385 0.01378249999999992 0.3179068 0.6244018 0.30531436333333334 0.6599067200000001 0.32031448333333334 0.6244018 0.36337965666666666 0.6599067200000001 0.3744400666666667


In [127]:
convert_kp_matrix_front_T(additional_kp_front(image_front_T_kp, image_front_I_kp)[0],288, 288)

[(50.0, 163.0),
 (47.0, 164.0),
 (47.0, 160.0),
 (51.0, 168.0),
 (52.0, 159.0),
 (71.0, 175.0),
 (77.0, 154.0),
 (103.0, 177.0),
 (106.0, 158.0),
 (134.0, 180.0),
 (84.0, 162.0),
 (121.0, 174.0),
 (123.0, 161.0),
 (169.0, 175.0),
 (173.0, 163.0),
 (209.0, 178.0),
 (213.0, 170.0),
 (87.0, 179.0),
 (92.0, 158.0),
 (88.0, 180.0),
 (92.0, 190.0),
 (105.0, 180.0),
 (108.0, 190.0)]

In [128]:
convert_kp_matrix_front_T(additional_kp_front(image_front_T_kp, image_front_I_kp)[1],288, 288)

[(39.0, 165.0),
 (36.0, 167.0),
 (35.0, 162.0),
 (38.0, 169.0),
 (38.0, 160.0),
 (57.0, 174.0),
 (57.0, 154.0),
 (75.0, 175.0),
 (74.0, 148.0),
 (68.0, 173.0),
 (74.0, 143.0),
 (114.0, 168.0),
 (113.0, 156.0),
 (161.0, 169.0),
 (161.0, 156.0),
 (210.0, 171.0),
 (209.0, 161.0),
 (87.0, 176.0),
 (168.0, 156.0),
 (88.0, 180.0),
 (92.0, 190.0),
 (105.0, 180.0),
 (108.0, 190.0)]

In [129]:
convert_kp_matrix_profile(additional_kp_profile(image_profile_kp), 288,288)

[(50.0, 163.0),
 (47.0, 164.0),
 (47.0, 160.0),
 (51.0, 168.0),
 (52.0, 159.0),
 (71.0, 175.0),
 (77.0, 154.0),
 (103.0, 177.0),
 (106.0, 158.0),
 (134.0, 180.0),
 (84.0, 162.0),
 (121.0, 174.0),
 (123.0, 161.0),
 (169.0, 175.0),
 (173.0, 163.0),
 (209.0, 178.0),
 (213.0, 170.0),
 (87.0, 163.0),
 (168.0, 156.0),
 (88.0, 163.0),
 (92.0, 156.0),
 (105.0, 163.0),
 (108.0, 156.0)]

In [ ]:
ratio_mm_px = ratio_real_vs_pixel(19)

## Measurements for T-shirt
https://freesewing.org/designs/teagan/

In [ ]:
# ratio_mm_px =  ratio_real_vs_pixel(190, matrix_image_contouring, image_h):

In [109]:
#Front picture in T shape
def biceps_circumference(matrix_kp_front_converted, matrix_image_front_cont):
    #left biceps
    x1 = matrix_kp_conv[17][1]
    sum_left = np.sum(matrix_image_front_cont[:, x1])
    
    #right biceps
    x2 = matrix_kp_conv[18][1]
    sum_right = np.sum(matrix_image_front_cont[:, x2])
    
    #diameter left / right biceps "height"
    average_mm = ((sum_left + sum_right) / 2) * ratio_mm_px
    #biceps circumference in mm
    circumference_biceps = (average_mm) * math.pi
        
    return  circumference_biceps

In [ ]:
biceps_circumference()

In [ ]:
#Front picture in T shape + profile picture
def chest_circumference(matrix_kp_front_converted, matrix_image_front_cont, matrix_kp_profile_converted, matrix_image_profile_cont, ):
    #measure width (along axis x) for given y, on front picture
    y_front1 = matrix_kp_front_converted[19][1]
    y_front2 = matrix_kp_front_converted[20][1]
    y _average = (y_front1+y_front2)/2
    sum_x_front = np.sum(matrix_image_front_cont[y,:])
    
    #measure width (along axis x) for given y, on profile picture
    y_prof1 = matrix_kp_front_converted[19][1]
    y_prof2 = matrix_kp_front_converted[20][1]
    y _average = (y_prof1 + y_prof2)/2
    sum_x_prof = np.sum(matrix_image_front_cont[y,:])
    
    #chest circumference in mm
    circumference_chest = (sum_x_front + sum_x_prof) * 2 * ratio_mm_px
    
    return circumference_chest

In [ ]:
#To be reviewed
#Current solution is distance between shoulder (# 5 / 6) and waist (# 21 / 22) with a factor 1.2
def hps_to_waist_back(matrix_kp_front_converted, matrix_image_front_cont):
    a = (matrix_kp_front_converted[5][0] - matrix_kp_front_converted[21][0]) + (matrix_kp_front_converted[6][0] - matrix_kp_front_converted[22][0])/2
    #convert in mm and apply factor of 1.2
    distance = a * 1.2 * ratio_mm_px
    
    return distance
    

In [ ]:
#Front picture in T shape + Profile picture
def hips_circumference(matrix_kp_front_converted, matrix_image_front_cont, matrix_kp_profile_converted, matrix_image_profile_cont):
    
    #measure width (along axis x) for given y, on front picture
    y_front1 = matrix_kp_front_converted[11][1]
    y_front2 = matrix_kp_front_converted[12][1]
    y _average = (y_front1+y_front2)/2
    sum_x_front = np.sum(matrix_image_front_cont[y,:])
    
    #measure width (along axis x) for given y, on profile picture
    y_prof1 = matrix_kp_front_converted[11][1]
    y_prof2 = matrix_kp_front_converted[12][1]
    y _average = (y_prof1 + y_prof2)/2
    sum_x_prof = np.sum(matrix_image_front_cont[y,:])
    
    #hips circumference in mm
    hips_circ = (sum_x_front + sum_x_prof) * 2 * ratio_mm_px
    
    return hips_circ

In [ ]:
#Profile picture
def neck_circumference( matrix_kp_profile_converted, matrix_image_profile_cont):
    #Estimated neck position on y located at 1/3 above shoulders vs ears
    neck_y = abs(matrix_kp_profile_converted[5][0] - matrix_kp_profile_converted[3][0])
    #sum over x considering neck_y = a
    sum_x = np.sum(matrix_image_profile_cont[neck_y,:])
    
    #neck circumference in mm
    neck_circ = (sum_x * ratio_mm_px) * math.pi
    
    return neck_circ
    

In [ ]:
#Front picture in T shape
#Consider distance (on x) between shoulder kp and ear kp, plus half width of biceps
def shoulder_slope(matrix_kp_front_converted, matrix_image_front_cont):
    #left biceps
    x1 = matrix_kp_front_converted[17][1]
    sum_left = np.sum(matrix_image_front_cont[:, x1])
    
    #distance (on x) between kp for shoulder / ear
    a = abs(matrix_kp_front_converted[5][1] - matrix_kp_front_converted[3][1]) + abs(matrix_kp_front_converted[4][1] - matrix_kp_front_converted[6][1]) / 2
    
    #overall distance in mm 
    total_distance = distance + sum_left/2 * ratio_mm_px
    
    return total_distance


In [ ]:
#Front picture in I shape
def shoulder_to_shoulder( matrix_kp_front_converted, matrix_image_front_cont ):
    #average y for shoulders kp
    x1 =( matrix_kp_front_converted[5][1] + matrix_kp_front_converted[6][1] ) / 2
    #distance (on x) between shoulders 
    sum_shoulders = np.sum(matrix_image_front_cont[x1, :])
    #convert distance in mm
    distance = sum_shoulders * ratio_mm_px
    
    return distance 
    

In [ ]:
#Front picture in T shape + Profile picture
def waist_circumference(matrix_kp_front_converted, matrix_image_front_cont, matrix_kp_profile_converted, matrix_image_profile_cont):
    
    #measure width (along axis x) for given y, on front picture
    y_front1 = matrix_kp_front_converted[21][0]
    y_front2 = matrix_kp_front_converted[22][0]
    y _average = (y_front1+y_front2)/2
    sum_x_front = np.sum(matrix_image_front_cont[y,:])
    
    #measure width (along axis x) for given y, on profile picture
    y_prof1 = matrix_kp_profile_converted[11][0]
    y_prof2 = matrix_kp_profile_converted[12][0]
    y _average = (y_prof1 + y_prof2)/2
    sum_x_prof = np.sum(matrix_image_front_cont[y,:])
    
    #waist circumference in mm
    waist_circumference = (sum_x_front + sum_x_prof) * 2 * ratio_mm_px
    
    return waist_circumference


In [ ]:
#Front picture in T shape
def waist_to_hips(matrix_kp_front_converted, matrix_image_front_cont):
    #average distance (on y) between waist(21,22) and hips(11,12) with correction factor of 1.2
    a = abs(matrix_kp_front_converted[22][1] - matrix_kp_front_converted[12][1])+abs(matrix_kp_front_converted[21][1] - matrix_kp_front_converted[11][1]) / 2
    #convert in mm and apply factor of 1.2
    distance = a * 1.2 * ratio_mm_px
    
    return distance


In [1]:
A = [[0.13626342, 0.57170576, 0.5406235 ],
       [0.12581952, 0.58031356, 0.5950187 ],
       [0.12306002, 0.5641533 , 0.5945765 ],
       [0.1330445 , 0.58676374, 0.46940276],
       [0.1319304 , 0.55489534, 0.48606545],
       [0.19684711, 0.6048043 , 0.37372732],
       [0.19630507, 0.53479004, 0.30953246],
       [0.26008302, 0.6065161 , 0.42267993],
       [0.25835255, 0.51460385, 0.49753478],
       [0.23563783, 0.59914243, 0.28201967],
       [0.25673127, 0.49728453, 0.4280582 ],
       [0.3964715 , 0.5841227 , 0.5168887 ],
       [0.3921252 , 0.54193866, 0.47156578],
       [0.5595846 , 0.58796036, 0.54856455],
       [0.5583141 , 0.54261553, 0.52169836],
       [0.7302867 , 0.5933529 , 0.4042293 ],
       [0.7241063 , 0.5598221 , 0.30607915]]


In [44]:
m = np.array([(1.5,2,3), (4,5,6), (10,-8,-4), (12,-4,-1)])
m

array([[ 1.5,  2. ,  3. ],
       [ 4. ,  5. ,  6. ],
       [10. , -8. , -4. ],
       [12. , -4. , -1. ]])

In [45]:
m[1,:]

array([4., 5., 6.])

In [41]:
sum(m[:,1])

-5.0

In [35]:
m.shape[0]

4

In [36]:
bb = range(0,11)
bb

range(0, 11)

In [37]:
for i in bb:
    print(i)

0
1
2
3
4
5
6
7
8
9
10
